In [ ]:
import requests
from bs4 import BeautifulSoup
import io
import zipfile

In [ ]:
SERVICE_URL = "https://batch.geocoder.ls.hereapi.com/6.2/jobs"
API_KEY = "YOUR_API_KEY"
job_id = None

In [ ]:
with open('data2geocode.txt', 'rb') as file:
  print(file.read())

b'recId|searchText\n0001|Invalidenstra\xc3\x9fe 116 10115 Berlin\n0002|Am Kronberger Hang 8 65824 Schwalbach\n0003|425 W Randolph St Chicago IL 60606\n0004|One Main Street Cambridge MA 02142\n0005|200 S Mathilda Ave Sunnyvale CA 94086'


In [ ]:
def start():
  """
  Запуск операции геокодирования.
  Сохранение ID работы для проверки статуса.
  """
  file = open('data2geocode.txt', 'rb')

  params = {
      "action": "run",
      "apiKey": API_KEY,
      "politicalview": "RUS",
      "gen": 9,
      "maxresults": "1",
      "header": "true",
      "indelim": "|",
      "outdelim": ";",
      "outcols": "displayLatitude,displayLongitude,locationLabel,houseNumber,street,district,city,postalCode,county,state,country",
      "outputcombined": "true",
  }

  response = requests.post(SERVICE_URL, params=params, data=file)

  parsedXMLResponse = BeautifulSoup(response.text, "lxml")
  job_id = parsedXMLResponse.find('requestid').get_text()
  print(job_id)
  
  file.close()

In [ ]:
start()

dHs52Uj1h9LwxoXIcmygxOIsP3IzRjwo


In [ ]:
def status(jobId):
  """
  Проверка статуса работы сервиса геокодирования.
  """
  statusURL = SERVICE_URL + "/" + jobId

  params = {
      "action": "status",
      "apiKey": API_KEY
  }

  response = requests.get(statusURL, params=params)
  print(response.text)

In [ ]:
status('dHs52Uj1h9LwxoXIcmygxOIsP3IzRjwo')

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><ns2:SearchBatch xmlns:ns2="http://www.navteq.com/lbsp/Search-Batch/1"><Response><MetaInfo><RequestId>dHs52Uj1h9LwxoXIcmygxOIsP3IzRjwo</RequestId></MetaInfo><Status>completed</Status><JobStarted>2021-06-03T12:30:26.000Z</JobStarted><JobFinished>2021-06-03T12:30:28.000Z</JobFinished><TotalCount>5</TotalCount><ValidCount>5</ValidCount><InvalidCount>0</InvalidCount><ProcessedCount>5</ProcessedCount><PendingCount>0</PendingCount><SuccessCount>5</SuccessCount><ErrorCount>0</ErrorCount></Response></ns2:SearchBatch>


In [ ]:
def result(job_id):
  """
  Скачивание результатов геокодирования.
  """
  resultURL = f'{SERVICE_URL}/{job_id}/result'

  params = {
      "apiKey": API_KEY
  }

  response = requests.get(resultURL, params=params)

  zipResult = zipfile.ZipFile(io.BytesIO(response.content))
  zipResult.extractall()
  print(response.url)

In [ ]:
result('dHs52Uj1h9LwxoXIcmygxOIsP3IzRjwo')

https://batch.geocoder.ls.hereapi.com/6.2/jobs/dHs52Uj1h9LwxoXIcmygxOIsP3IzRjwo/result?apiKey=idhlDg7Y08LoZwNQ6UCPfq5bsxtrbLMpJZi58GyIcIQ
